## PCA (Principal component analysis)
### What is PCA?
PCA is commonly known as a dimensionality reduction algorithm, but it can also be used to visualize data (as high dimensions of data can be hard to visualize)

### Great, what are the steps to do PCA?
1. Standardize the range of continuous initial variables
2. Compute the covariance matrix to identify correlations
3. Compute the eigenvectors and eigenvalues of the covariance matrix to identify the principal components
4. Create a feature vector to decide which principal components to keep, basically keeping k eigenvalues
5. Recast the data along the principal components axes

### What are eigenvalues and eigenvectors?
Eigenvalues is a nonzero vector that changes at most by a scalar factor when that linear transformation is applied to it. The corresponding eigenvalue is the factor by which the eigenvector is scaled.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()